<a href="https://colab.research.google.com/github/yujin-jo/mini-project-team1/blob/main/analysis_04_%EB%B0%B0%EA%B8%89%EC%82%AC%EB%B9%84%EC%A4%91%EB%B3%80%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 기본 세팅
- 깃허브 클론 
- 드라이브 마운트 
- 파일 임포트

In [ ]:
!git clone https://github.com/yujin-jo/mini-project-team1.git

Cloning into 'mini-project-team1'...
remote: Enumerating objects: 567, done.
remote: Counting objects: 100% (567/567), done.
remote: Compressing objects: 100% (531/531), done.
remote: Total 567 (delta 160), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (567/567), 731.53 KiB | 5.30 MiB/s, done.
Resolving deltas: 100% (160/160), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd 
import numpy as   np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

In [10]:
movies = pd.read_csv("/content/mini-project-team1/1. Crawling/data/KOBIS_TOT.csv")
movies["개봉일"] = pd.to_datetime(movies["개봉일"])
movies.head(2)

,순번,개봉연도,영화명,감독,배급사,개봉일,국적,전국스크린수,전국매출액,전국관객수,장르,등급,영화구분
0,1,2011,트랜스포머 3,마이클 베이,(주)씨제이이엔엠,2011-06-29,미국,1409,74841350500,7784807,액션,12세이상관람가,일반영화
1,2,2011,최종병기 활,김한민,롯데쇼핑㈜롯데엔터테인먼트,2011-08-10,한국,615,55808856500,7466976,사극,15세이상관람가,일반영화


---
# I. 가설 설정과 분석 배경

1. 가설  :  2011년 ~ 2020년까지 국내 영화 배급사에서 CJ E&M의 비중은 연도 별로 감소했는가? 
2. 가설을 선택한 이유 : 
    - 기존에는 국내 영화의 경우 CJ E&M이 독점했왔음. 
    - 다른 신생 배급사들은 없었을까 궁금했음 
    -  기존에 있던 영세 배급사들은 성장하거나 망했을까 궁금했고, 여전히 CJ E&M이 영화 산업을 지배하고 있는지 확인하고 싶었음 

# II. 분석 방향 
1. 활용 데이터 
  - 영화 개봉연도 : 연도 별로 영화 수 집계 필요 
  - 영화 배급사 : 영화 별 배급사 정보 (KOBIS 참고) 
  - 영화 수 : 연도 별, 배급사 별로 집계한 영화 수

2. 분석 방법론 
  - 시계열 분석 활용 (2011 ~ 2020년 영화 정보) 
  - 영화 배급사의 종류가 연도 별로 어떻게 달라졌는지 확인  (중소 배급사 등 배급사가 늘었는지 확인하기 위함)
  - 개봉 연도와 제작사 별로 영화 편수를 카운트한다. 
  - 연도 별로 제작사 별 비중을 계산하여 추이를 확인한다. 

3. 시각화 
  - 비율 바 차트




# III. 분석 과정  

## 1. 데이터 테이블 분석
- 2011년 ~ 2020년까지의 총 3,375개의 영화 데이터가 들어감 
- movie 테이블에 들어간 정보를 movies.info()로 파악 
- 영화 배급사의 종류는 '씨제이이엔엠', '롯데쇼핑(주)롯데엔터테인먼트', '유니버설픽쳐스인터네셔널 코리아(유)', '(주)넥스트엔터테인먼트월드 (NEW)', '(주) 팝엔터테인먼트',

In [17]:
# 영화 기본 정보 확인 
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3375 entries, 0 to 3374
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   순번      3375 non-null   int64         
 1   개봉연도    3375 non-null   int64         
 2   영화명     3375 non-null   object        
 3   감독      3375 non-null   object        
 4   배급사     3374 non-null   object        
 5   개봉일     3375 non-null   datetime64[ns]
 6   국적      3375 non-null   object        
 7   전국스크린수  3375 non-null   int64         
 8   전국매출액   3375 non-null   int64         
 9   전국관객수   3375 non-null   int64         
 10  장르      3375 non-null   object        
 11  등급      3375 non-null   object        
 12  영화구분    3375 non-null   object        
dtypes: datetime64[ns](1), int64(5), object(7)
memory usage: 342.9+ KB


In [21]:
# 영화 배급사 종류 확인 
movies['배급사'].value_counts()

(주)씨제이이엔엠                                         215
롯데쇼핑㈜롯데엔터테인먼트                                     165
유니버설픽쳐스인터내셔널 코리아(유)                               148
(주)넥스트엔터테인먼트월드(NEW)                               138
(주)팝엔터테인먼트                                        135
                                                 ... 
(주)스튜디오 보난자                                         1
(주)패뷸러스                                             1
(주)더블앤조이픽쳐스,(주)브리즈픽처스                               1
롯데쇼핑㈜롯데시네마                                          1
(주)넥스트엔터테인먼트월드(NEW),KT&G 상상마당 영화사업팀 컴퍼니에스에스(주)      1
Name: 배급사, Length: 434, dtype: int64